In [5]:
import boto3
import os
from botocore.config import Config

# 1. 你的 PC 端 MinIO 配置
s3 = boto3.client(
    's3',
    endpoint_url='http://127.0.0.1:9000',
    aws_access_key_id='admin',
    aws_secret_access_key='password123',
    config=Config(s3={'addressing_style': 'path'})
)

bucket_name = 'nnpy'
tailscale_ip = '100.77.8.124'
urls_file = 'urls.txt'

print(f"--- 正在深度扫描 {bucket_name} 桶内所有文件 ---")

try:
    # 使用分页器绕过 1000 个文件的限制
    paginator = s3.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=bucket_name)
    
    total_count = 0
    with open(urls_file, 'w') as f:
        for page in pages:
            if 'Contents' in page:
                for obj in page['Contents']:
                    file_key = obj['Key']
                    
                    # 过滤掉 MinIO 的系统元数据文件夹
                    if '.minio.sys' in file_key:
                        continue
                    # 仅保留 .tar.gz 压缩包（每批 64 个 npy 的打包文件）
                    if not file_key.endswith('.tar.gz'):
                        continue
                        
                    # 生成签名链接
                    url = s3.generate_presigned_url(
                        'get_object',
                        Params={'Bucket': bucket_name, 'Key': file_key},
                        ExpiresIn=86400  # 有效期 24 小时
                    )
                    
                    # 替换为 Tailscale IP 并写入
                    f.write(url.replace('127.0.0.1', tailscale_ip) + '\n')
                    total_count += 1
                    
                    if total_count % 1000 == 0:
                        print(f"已扫描 {total_count} 个文件...")

    if total_count == 0:
        print(f"❌ 错误：未发现有效文件。")
    else:
        print(f"\n✅ 完成！共生成 {total_count} 个文件的链接：{urls_file}")

except Exception as e:
    print(f"❌ 出错: {e}")

--- 正在深度扫描 nnpy 桶内所有文件 ---
已扫描 1000 个文件...
已扫描 2000 个文件...
已扫描 3000 个文件...
已扫描 4000 个文件...
已扫描 5000 个文件...
已扫描 6000 个文件...
已扫描 7000 个文件...
已扫描 8000 个文件...
已扫描 9000 个文件...

✅ 完成！共生成 9818 个文件的链接：urls.txt


In [2]:
pip install boto3

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 14.6/14.6 MB 106.4 MB/s eta 0:00:00

   ---------- ----------------------------- 1/4 [botocore]
   ---------- ----------------------------- 1/4 [botocore]
   ---------- ----------------------------- 1/4 [botocore]
   ---------- ----------------------------- 1/4 [botocore]
   ---------- ----------------------------- 1/4 [botocore]
   ---------- ----------------------------- 1/4 [botocore]
   ---------- ----------------------------- 1/4 [botocore]
   ---------- ----------------------------- 1/4 [botocore]
   ---------- ----------------------------- 1/4 [botocore]
   -------------------- ------------------- 2/4 [s3transfer]
   ------------------------------ --------- 3/4 [boto3]
   ---------------------------------------- 4/4 [boto3]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
"""
将 D:\deepLesion\npy 下的 .npy 文件按每 64 个打包成 .tar.gz，保存到 D:\deepLesion\64pack
"""
import os
import tarfile
from pathlib import Path

SRC_DIR = r"D:\deepLesion\npy"
OUT_DIR = r"D:\deepLesion\64pack"
BATCH_SIZE = 64

Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

# 获取所有 .npy 文件并排序
npy_files = sorted(Path(SRC_DIR).glob("*.npy"))
total = len(npy_files)
print(f"找到 {total} 个 .npy 文件")

num_packs = (total + BATCH_SIZE - 1) // BATCH_SIZE
for pack_idx in range(num_packs):
    start = pack_idx * BATCH_SIZE
    end = min(start + BATCH_SIZE, total)
    batch = npy_files[start:end]
    
    archive_name = f"batch_{pack_idx:05d}.tar.gz"
    archive_path = Path(OUT_DIR) / archive_name
    
    with tarfile.open(archive_path, "w:gz") as tf:
        for f in batch:
            tf.add(str(f), arcname=f.name)
    
    print(f"  {archive_name}: {len(batch)} 个文件")

print(f"\n完成，共生成 {num_packs} 个压缩包，保存在 {OUT_DIR}")

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\hrole\AppData\Local\Temp\ipykernel_274032\600153143.py:1: SyntaxWarning: invalid escape sequence '\d'
  """


找到 9816 个 .npy 文件
  batch_00000.tar.gz: 64 个文件
  batch_00001.tar.gz: 64 个文件
  batch_00002.tar.gz: 64 个文件
  batch_00003.tar.gz: 64 个文件
  batch_00004.tar.gz: 64 个文件
  batch_00005.tar.gz: 64 个文件
  batch_00006.tar.gz: 64 个文件
  batch_00007.tar.gz: 64 个文件
  batch_00008.tar.gz: 64 个文件
  batch_00009.tar.gz: 64 个文件
  batch_00010.tar.gz: 64 个文件
  batch_00011.tar.gz: 64 个文件
  batch_00012.tar.gz: 64 个文件
  batch_00013.tar.gz: 64 个文件
  batch_00014.tar.gz: 64 个文件
  batch_00015.tar.gz: 64 个文件
  batch_00016.tar.gz: 64 个文件
  batch_00017.tar.gz: 64 个文件
  batch_00018.tar.gz: 64 个文件
  batch_00019.tar.gz: 64 个文件
  batch_00020.tar.gz: 64 个文件
  batch_00021.tar.gz: 64 个文件
  batch_00022.tar.gz: 64 个文件
  batch_00023.tar.gz: 64 个文件
  batch_00024.tar.gz: 64 个文件
  batch_00025.tar.gz: 64 个文件
  batch_00026.tar.gz: 64 个文件
  batch_00027.tar.gz: 64 个文件
  batch_00028.tar.gz: 64 个文件
  batch_00029.tar.gz: 64 个文件
  batch_00030.tar.gz: 64 个文件
  batch_00031.tar.gz: 64 个文件
  batch_00032.tar.gz: 64 个文件
  batch_00033.tar.gz: 64 